In [1]:
#| default_exp ml_models


In [3]:
#| export
import numpy as np
from typing import Dict, Any, List, Optional
from transformers import pipeline
from snowflake.ml.modeling.distributors.xgboost import XGBEstimator, XGBScalingConfig
from snowflake.ml.data.data_connector import DataConnector
from snowflake.snowpark.dataframe import DataFrame


/Users/jdemlow/miniconda3/envs/sfml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'snowflake.ml.modeling.distributors'

In [ ]:
#| export
class ModelPredictor:
    """Zero-shot classification model for review quality prediction."""
    
    def __init__(self, model_name: str = "facebook/bart-large-mnli",
                 candidate_labels: List[str] = None):
        """Initialize the model predictor.
        
        Args:
            model_name: HuggingFace model name for zero-shot classification
            candidate_labels: List of labels for classification
        """
        if candidate_labels is None:
            candidate_labels = [
                'detailed with specific information and experience', 
                'basic accurate information', 
                'generic brief with no details'
            ]
        
        self.candidate_labels = candidate_labels
        self.classifier = pipeline("zero-shot-classification", model=model_name)

    def __call__(self, batch: Dict[str, Any]) -> Dict[str, Any]:
        """Predict review quality for a batch of reviews.
        
        Args:
            batch: Dictionary containing review text data
            
        Returns:
            Updated batch with prediction results
        """
        resp = self.classifier(batch["REVIEW_TEXT"].tolist(), self.candidate_labels)

        # Handle both resp and batch results
        if isinstance(resp, dict):
            raise ValueError(f"Expected batch response, got {resp} for batch {batch['REVIEW_TEXT']}")
            
        # Add results to batch
        batch["REVIEW_QUALITY"] = np.array([
            result["labels"][np.argmax(result["scores"])] for result in resp
        ])
        
        return batch


In [ ]:
#| export
def create_xgb_model(config: Dict[str, Any] = None) -> XGBEstimator:
    """Create and configure XGBoost estimator.
    
    Args:
        config: Configuration dictionary with model parameters
        
    Returns:
        Configured XGBEstimator
    """
    if config is None:
        params = {
            "eta": 0.1,
            "max_depth": 8,
            "min_child_weight": 100,
            "tree_method": "hist",
        }
        n_estimators = 50
    else:
        ml_config = config.get('ml', {}).get('models', {}).get('purchase_prediction', {})
        params = {
            "eta": ml_config.get('learning_rate', 0.1),
            "max_depth": ml_config.get('max_depth', 8),
            "min_child_weight": 100,
            "tree_method": "hist",
        }
        n_estimators = ml_config.get('n_estimators', 50)
    
    scaling_config = XGBScalingConfig(use_gpu=False)
    
    estimator = XGBEstimator(
        n_estimators=n_estimators,
        objective="reg:squarederror",
        params=params,
        scaling_config=scaling_config,
    )
    
    return estimator
